## Setup

Import necessary libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
# To enable import from future_encoders.py
import sys
sys.path.append(os.path.join(os.getcwd(), '../'))

%matplotlib inline

np.random.seed(42)

## Read Data

In [5]:
DATASETS_PATH = os.path.join(os.getcwd(), '../', 'datasets')
TITANIC_PATH = os.path.join(DATASETS_PATH, 'titanic')

def load_titanic_data(filename):
    target_path = os.path.join(TITANIC_PATH, filename)
    return pd.read_csv(target_path)

train_raw = load_titanic_data('train.csv')
test_raw = load_titanic_data('test.csv')

train_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preapare Data

In [6]:
train_raw.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Drop columns with 50% missing value, get_missing_values_table function got from https://towardsdatascience.com/a-complete-machine-learning-walk-through-in-python-part-one-c62152f39420

In [7]:
# Function to calculate missing values by column# Funct 
def get_missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

missing_values_table = get_missing_values_table(train_raw)
missing_values_table

Your selected dataframe has 12 columns.
There are 3 columns that have missing values.


,Missing Values,% of Total Values
Cabin,687,77.1
Age,177,19.9
Embarked,2,0.2


In [8]:
cols_with_over_50_missing = missing_values_table[missing_values_table["% of Total Values"] > 50].index
train_useful = train_raw.drop(list(cols_with_over_50_missing), axis=1)
train_useful.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


Here we split out the features and target

In [9]:
X_train = train_useful.drop('Survived', axis=1)
y_train = train_useful['Survived'].copy()

## Create Pipeline

The Name, PassengerId and Ticket columns are skipped due to too many categories

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, Imputer
from future_encoders import OneHotEncoder

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes):
        self.attributes = attributes
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attributes]
    
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X], index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

cat_cols = ['Sex', 'Embarked', 'Pclass']
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']

num_pipeline = Pipeline([
    ('num_selector', DataFrameSelector(num_cols)),
    ('median_imputer', Imputer(strategy='median')),
    ('std_scaler', StandardScaler()),
])

cat_pipeline = Pipeline([
    ('cat_selector', DataFrameSelector(cat_cols)),
    ('most_frequent_imputer', MostFrequentImputer()),
    ('one_hot_encoder', OneHotEncoder(sparse=False)),
])

data_preparation_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

X_train_prep = data_preparation_pipeline.fit_transform(X_train)

print("X_train_prep shape: ", X_train_prep.shape)
print("X_train_prep single row: ", X_train_prep[0])

X_train_prep shape:  (891, 12)
X_train_prep single row:  [-0.56573646  0.43279337 -0.47367361 -0.50244517  0.          1.
  0.          0.          1.          0.          0.          1.        ]


## Model Evaluation

Import metrics functions for evalutaion.

In [32]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

def show_top_k_cv_params(cv_model, k):
    cv_results = cv_model.cv_results_
    score_param_list = list(zip(cv_results['mean_test_score'], cv_results['params']))
    sorted_score_param_list = sorted(score_param_list, key=lambda tup: tup[0], reverse=True)[:k]
    for score_param in sorted_score_param_list:
        print(score_param)

def show_best_score_results(cv_model, X_features, y_label):
    y_predict = cv_model.best_estimator_.predict(X_features)
    cv_scores = cross_val_score(cv_model.best_estimator_, X_features, y_label, cv=10)
    print("f1 score: ", f1_score(y_label, y_predict))
    print("precision score: ", precision_score(y_label, y_predict))
    print("recall score: ", recall_score(y_label, y_predict))
    print("accuracy score: ", accuracy_score(y_label, y_predict))
    print("roc_auc score: ", roc_auc_score(y_label, y_predict))
    print("cross validation score: ", cv_scores.mean())

### K-neighbors

In [12]:
kn_params = {
    'weights': ["uniform", "distance"],
    'n_neighbors': [3, 4, 5, 6, 7],
    'leaf_size': [3, 10, 30, 100, 300, 1000, 3000]
}
kn_cls = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_distributions=kn_params,
    n_iter=50,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
kn_cls.fit(X_train_prep, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    0.8s finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
          fit_params=None, iid=True, n_iter=50, n_jobs=-1,
          param_distributions={'leaf_size': [3, 10, 30, 100, 300, 1000, 3000], 'weights': ['uniform', 'distance'], 'n_neighbors': [3, 4, 5, 6, 7]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=1)

#### K-neighbors results

In [10]:
show_top_k_cv_params(kn_cls, 5)
show_best_score_results(kn_cls, X_train_prep, y_train)

(0.8013468013468014, {'n_neighbors': 6, 'weights': 'uniform', 'leaf_size': 10})
(0.8013468013468014, {'n_neighbors': 6, 'weights': 'uniform', 'leaf_size': 100})
(0.8013468013468014, {'n_neighbors': 6, 'weights': 'uniform', 'leaf_size': 30})
(0.8002244668911336, {'n_neighbors': 6, 'weights': 'uniform', 'leaf_size': 300})
(0.8002244668911336, {'n_neighbors': 6, 'weights': 'uniform', 'leaf_size': 1000})
f1 score:  0.7845659163987139
precision score:  0.8714285714285714
recall score:  0.7134502923976608
accuracy score:  0.8496071829405163
cross validation score:  0.809170355237771


### Random Forest

In [11]:
rf_params = {
    'bootstrap': [True, False],
    'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
}
rf_cls = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_distributions=rf_params,
    n_iter=100,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
rf_cls.fit(X_train_prep, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   53.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  7.1min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'bootstrap': [True, False], 'min_samples_split': [2, 5, 10], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'min_samples_leaf': [1, 2, 4], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'max_features': ['auto', 'sqrt']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='accuracy', verbose=1)

#### Random Forest results

In [12]:
show_top_k_cv_params(rf_cls, 5)
show_best_score_results(rf_cls, X_train_prep, y_train)

(0.8282828282828283, {'bootstrap': True, 'min_samples_split': 10, 'n_estimators': 1800, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90})
(0.8282828282828283, {'bootstrap': True, 'min_samples_split': 10, 'n_estimators': 1400, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20})
(0.8282828282828283, {'bootstrap': True, 'min_samples_split': 10, 'n_estimators': 1800, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 20})
(0.8282828282828283, {'bootstrap': True, 'min_samples_split': 10, 'n_estimators': 1400, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 50})
(0.8282828282828283, {'bootstrap': True, 'min_samples_split': 10, 'n_estimators': 1200, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 100})
f1 score:  0.8713178294573645
precision score:  0.9273927392739274
recall score:  0.8216374269005848
accuracy score:  0.9068462401795735
cross validation score:  0.8283855408012712


### XGBoost 

In [38]:
from xgboost import XGBClassifier

xgb_params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.001, 0.003, 0.005, 0.01, 0.03, 0.05],
    'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400]
}
xgb_cls = GridSearchCV(
    XGBClassifier(),
    param_grid=xgb_params,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)
xgb_cls.fit(X_train_prep, y_train)

Fitting 5 folds for each of 126 candidates, totalling 630 fits


/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

...

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

[Parallel(n_jobs=-1)]: Done 630 out of 630 | elapsed:  3.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.001, 0.003, 0.005, 0.01, 0.03, 0.05], 'max_depth': [3, 5, 7], 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [39]:
show_top_k_cv_params(xgb_cls, 5)
show_best_score_results(xgb_cls, X_train_prep, y_train)

(0.8372615039281706, {'learning_rate': 0.003, 'max_depth': 7, 'n_estimators': 1200})
(0.835016835016835, {'learning_rate': 0.003, 'max_depth': 7, 'n_estimators': 1000})
(0.835016835016835, {'learning_rate': 0.005, 'max_depth': 7, 'n_estimators': 600})
(0.8338945005611672, {'learning_rate': 0.003, 'max_depth': 7, 'n_estimators': 1400})
(0.8338945005611672, {'learning_rate': 0.01, 'max_depth': 7, 'n_estimators': 400})


/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: Deprec

f1 score:  0.8526645768025078
precision score:  0.918918918918919
recall score:  0.7953216374269005
accuracy score:  0.8945005611672279
roc_auc score:  0.8758028952161825
cross validation score:  0.830608330495971


/Users/howard/anaconda3/envs/mlbook/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Evaluate on Test Set

We already found the best model with tuned hyper parameter which is rf_cls, let's output our predictions!

In [13]:
X_test_prep = data_preparation_pipeline.transform(test_raw)
y_test_pred = rf_cls.best_estimator_.predict(X_test_prep)
result_array = [test_raw['PassengerId'], pd.Series(y_test_pred, name='Survived')]
result_df = pd.concat(result_array, axis=1)
result_df.to_csv('result.csv', index=False)